In [1]:
!pip install opendatasets
import opendatasets as od
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
encoder_prod = LabelEncoder()
encoder_cpu = LabelEncoder()
encoder_gpu = LabelEncoder()
encoder_weight = LabelEncoder()
encoder_price = LabelEncoder()

**Целевая переменная Price_euros, задача регрессии**

In [3]:
#1ae760cc24c13c57f64d82570894a87b

In [5]:
od.download('https://www.kaggle.com/competitions/laptop-price-prediction-misis/data')   

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nitnatsn0k
Your Kaggle Key: ··········


100%|██████████| 29.1k/29.1k [00:00<00:00, 2.45MB/s]


Extracting archive ./laptop-price-prediction-misis/laptop-price-prediction-misis.zip to ./laptop-price-prediction-misis


# **работа с TRAIN**

In [15]:
train = pd.read_csv('/content/laptop-price-prediction-misis/Price_euros_train.csv')
train

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
1,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
2,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60
3,6,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.1kg,400.00
4,7,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.2GHz,16GB,256GB Flash Storage,Intel Iris Pro Graphics,Mac OS X,2.04kg,2139.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,1315,Asus,X556UJ-XO044T (i7-6500U/4GB/500GB/GeForce,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,4GB,500GB HDD,Nvidia GeForce 920M,Windows 10,2.2kg,720.32
1104,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,638.00
1105,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1106,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108 entries, 0 to 1107
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         1108 non-null   int64  
 1   Company           1108 non-null   object 
 2   Product           1108 non-null   object 
 3   TypeName          1108 non-null   object 
 4   Inches            1108 non-null   float64
 5   ScreenResolution  1097 non-null   object 
 6   Cpu               1108 non-null   object 
 7   Ram               1108 non-null   object 
 8   Memory            1108 non-null   object 
 9   Gpu               1108 non-null   object 
 10  OpSys             1055 non-null   object 
 11  Weight            1086 non-null   object 
 12  Price_euros       1108 non-null   float64
dtypes: float64(2), int64(1), object(10)
memory usage: 112.7+ KB


In [17]:
#анализируем признаки 
print(train['laptop_ID'].value_counts(), "\n", len(train['laptop_ID'].unique()))
 # у каждого ноутбука свой ID тогда можем удалить данный признак, от остальных параметров точно зависит цена, так их удалять не будем
train = train.drop(['laptop_ID'], axis = 1)

2       1
877     1
884     1
883     1
882     1
       ..
434     1
435     1
436     1
437     1
1320    1
Name: laptop_ID, Length: 1108, dtype: int64 
 1108


In [18]:
train.isna().sum()

Company              0
Product              0
TypeName             0
Inches               0
ScreenResolution    11
Cpu                  0
Ram                  0
Memory               0
Gpu                  0
OpSys               53
Weight              22
Price_euros          0
dtype: int64

**убираем из признаков Ram и Weight ненужные символы(GB и KG) и изменяем тип на int, чтобы модель работала уже c числами.**

In [19]:
train['Ram'] = train['Ram'].str.replace(r"[^\d\.]", "", regex=True)
train['Ram'] = pd.to_numeric(train['Ram'])

train['Weight'] = train['Weight'].str.replace(r"[^\d\.]", "", regex=True)
train['Weight'] = pd.to_numeric(train['Weight'])

train.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94
1,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45
2,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60
3,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4,500GB HDD,AMD Radeon R5,Windows 10,2.10,400.00
4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.2GHz,16,256GB Flash Storage,Intel Iris Pro Graphics,Mac OS X,2.04,2139.97


In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108 entries, 0 to 1107
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           1108 non-null   object 
 1   Product           1108 non-null   object 
 2   TypeName          1108 non-null   object 
 3   Inches            1108 non-null   float64
 4   ScreenResolution  1097 non-null   object 
 5   Cpu               1108 non-null   object 
 6   Ram               1108 non-null   int64  
 7   Memory            1108 non-null   object 
 8   Gpu               1108 non-null   object 
 9   OpSys             1055 non-null   object 
 10  Weight            1086 non-null   float64
 11  Price_euros       1108 non-null   float64
dtypes: float64(3), int64(1), object(8)
memory usage: 104.0+ KB


In [21]:
#заполним пропуски в ScreenResolution для дальнейшей работы
value1 = train['ScreenResolution'].mode()[0] #заполняем пропуски самым частым значением
train['ScreenResolution'] = train['ScreenResolution'].fillna(value1)

**Теперь нужно разделить некоторые признаки на несколько, например Cpu разделить на модель и герцовку**

In [22]:
#разделяем Cpu на Cpu_Hz и Cpu_Model
train['Cpu_Hz'] = train['Cpu'].apply(lambda x: x.split()[-1])
train['Cpu_Hz'] = train['Cpu_Hz'].str.replace(r"[^\d\.]", "", regex=True)
train['Cpu_Hz'] = pd.to_numeric(train['Cpu_Hz'])

train['Cpu_Model'] = train['Cpu'].apply(lambda x: ' '.join(x.split()[:-1]))

#разделяем ScreenResolution на Resolution (который разделим на 2 числовых значения ширина(width) и высота(height)), 
#а модель Screen отбросим так как она есть далеко не везде, а пропуски нам не нужны.
train['ScreenResolution'] = train['ScreenResolution'].astype(str)
train['Resolution'] = train['ScreenResolution'].apply(lambda x: x.split()[-1])

train['Width'] = train['Resolution'].apply(lambda x: x.split('x')[0])
train['Width'] = pd.to_numeric(train['Width'])

train['Height'] = train['Resolution'].apply(lambda x: ' '.join(x.split('x')[1:]))
train['Height'] = pd.to_numeric(train['Height'])


#разделяем Memory на Memory_size и Memory_type
train['Memory_Size'] = train['Memory'].apply(lambda x: x.split()[0])
train['Memory_Size'] = train['Memory_Size'].str.replace(r"[^\d\.]", "", regex=True)
train['Memory_Size'] = pd.to_numeric(train['Memory_Size'])

train['Memory_type'] = train['Memory'].apply(lambda x: ' '.join(x.split()[1:]))

#удаляем колонки Memory, Cpu, ScreenResolution.
train = train.drop(['Memory'], axis = 1)
train = train.drop(['Cpu'], axis = 1)                   #удаляем столбцы которые мы уже разделили
train = train.drop(['ScreenResolution'], axis = 1)
train = train.drop(['Resolution'], axis = 1) 
train 

,Company,Product,TypeName,Inches,Ram,Gpu,OpSys,Weight,Price_euros,Cpu_Hz,Cpu_Model,Width,Height,Memory_Size,Memory_type
0,Apple,Macbook Air,Ultrabook,13.3,8,Intel HD Graphics 6000,macOS,1.34,898.94,1.8,Intel Core i5,1440,900,128.0,Flash Storage
1,Apple,MacBook Pro,Ultrabook,15.4,16,AMD Radeon Pro 455,macOS,1.83,2537.45,2.7,Intel Core i7,2880,1800,512.0,SSD
2,Apple,MacBook Pro,Ultrabook,13.3,8,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,3.1,Intel Core i5,2560,1600,256.0,SSD
3,Acer,Aspire 3,Notebook,15.6,4,AMD Radeon R5,Windows 10,2.10,400.00,3.0,AMD A9-Series 9420,1366,768,500.0,HDD
4,Apple,MacBook Pro,Ultrabook,15.4,16,Intel Iris Pro Graphics,Mac OS X,2.04,2139.97,2.2,Intel Core i7,2880,1800,256.0,Flash Storage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,Asus,X556UJ-XO044T (i7-6500U/4GB/500GB/GeForce,Notebook,15.6,4,Nvidia GeForce 920M,Windows 10,2.20,720.32,2.5,Intel Core i7 6500U,1366,768,500.0,HDD
1104,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,4,Intel HD Graphics 520,Windows 10,1.80,638.00,2.5,Intel Core i7 6500U,1920,1080,128.0,SSD
1105,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,2,Intel HD Graphics,Windows 10,1.50,229.00,1.6,Intel Celeron Dual Core N3050,1366,768,64.0,Flash Storage
1106,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,6,AMD Radeon R5 M330,Windows 10,2.19,764.00,2.5,Intel Core i7 6500U,1366,768,1.0,HDD


In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108 entries, 0 to 1107
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1108 non-null   object 
 1   Product      1108 non-null   object 
 2   TypeName     1108 non-null   object 
 3   Inches       1108 non-null   float64
 4   Ram          1108 non-null   int64  
 5   Gpu          1108 non-null   object 
 6   OpSys        1055 non-null   object 
 7   Weight       1086 non-null   float64
 8   Price_euros  1108 non-null   float64
 9   Cpu_Hz       1108 non-null   float64
 10  Cpu_Model    1108 non-null   object 
 11  Width        1108 non-null   int64  
 12  Height       1108 non-null   int64  
 13  Memory_Size  1108 non-null   float64
 14  Memory_type  1108 non-null   object 
dtypes: float64(5), int64(3), object(7)
memory usage: 130.0+ KB


In [24]:
train.isna().sum()

Company         0
Product         0
TypeName        0
Inches          0
Ram             0
Gpu             0
OpSys          53
Weight         22
Price_euros     0
Cpu_Hz          0
Cpu_Model       0
Width           0
Height          0
Memory_Size     0
Memory_type     0
dtype: int64

In [25]:
#заполняем пропуски самыым часттым значением
value2 = train['OpSys'].mode()[0]
value3 = train['Weight'].mode()[0]

train['OpSys'] = train['OpSys'].fillna(value2)
train['Weight'] = train['Weight'].fillna(value3)

In [26]:
train.isna().sum()

Company        0
Product        0
TypeName       0
Inches         0
Ram            0
Gpu            0
OpSys          0
Weight         0
Price_euros    0
Cpu_Hz         0
Cpu_Model      0
Width          0
Height         0
Memory_Size    0
Memory_type    0
dtype: int64

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108 entries, 0 to 1107
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1108 non-null   object 
 1   Product      1108 non-null   object 
 2   TypeName     1108 non-null   object 
 3   Inches       1108 non-null   float64
 4   Ram          1108 non-null   int64  
 5   Gpu          1108 non-null   object 
 6   OpSys        1108 non-null   object 
 7   Weight       1108 non-null   float64
 8   Price_euros  1108 non-null   float64
 9   Cpu_Hz       1108 non-null   float64
 10  Cpu_Model    1108 non-null   object 
 11  Width        1108 non-null   int64  
 12  Height       1108 non-null   int64  
 13  Memory_Size  1108 non-null   float64
 14  Memory_type  1108 non-null   object 
dtypes: float64(5), int64(3), object(7)
memory usage: 130.0+ KB


In [29]:
#закодируем категориальеные переменные с помощью Labelencoder, чтобы не было несостыковок с кодированием test

train['Company'] = LabelEncoder().fit_transform(train['Company'])
train['TypeName'] = LabelEncoder().fit_transform(train['TypeName'])
train['OpSys'] = LabelEncoder().fit_transform(train['OpSys'])
train['Memory_type'] = LabelEncoder().fit_transform(train['Memory_type'])

train['Product'] = LabelEncoder().fit_transform(train['Product'])
train['Gpu'] = LabelEncoder().fit_transform(train['Gpu'])
train['Cpu_Model'] = LabelEncoder().fit_transform(train['Cpu_Model'])

train

,Company,Product,TypeName,Inches,Ram,Gpu,OpSys,Weight,Price_euros,Cpu_Hz,Cpu_Model,Width,Height,Memory_Size,Memory_type
0,1,269,4,13.3,8,49,8,1.34,898.94,1.8,49,1440,900,128.0,0
1,1,268,4,15.4,16,8,8,1.83,2537.45,2.7,63,2880,1800,512.0,5
2,1,268,4,13.3,8,57,8,1.37,1803.60,3.1,49,2560,1600,256.0,5
3,0,51,3,15.6,4,16,5,2.10,400.00,3.0,11,1366,768,500.0,2
4,1,268,4,15.4,16,58,3,2.04,2139.97,2.2,63,2880,1800,256.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,2,505,3,15.6,4,63,5,2.20,720.32,2.5,64,1366,768,500.0,2
1104,9,514,0,14.0,4,45,5,1.80,638.00,2.5,64,1920,1080,128.0,5
1105,9,176,3,14.0,2,38,5,1.50,229.00,1.6,30,1366,768,64.0,0
1106,7,2,3,15.6,6,19,5,2.19,764.00,2.5,64,1366,768,1.0,2


In [30]:
X = train.drop(['Price_euros'], axis=1)
Y = train['Price_euros']  

In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_st = scaler.fit_transform(X)
pd.DataFrame(X_st, columns = X.columns)

,Company,Product,TypeName,Inches,Ram,Gpu,OpSys,Weight,Cpu_Hz,Cpu_Model,Width,Height,Memory_Size,Memory_type
0,-1.362239,-0.035779,1.134816,-1.221219,-0.071167,-0.157065,3.119309,-1.068903,-0.976340,-0.472252,-0.927459,-0.602717,-0.568179,-2.019505
1,-1.362239,-0.042370,1.134816,0.258515,1.505896,-2.067927,3.119309,-0.321084,0.795916,0.344332,2.027407,2.603663,1.625249,0.392234
2,-1.362239,-0.042370,1.134816,-1.221219,-0.071167,0.215786,3.119309,-1.023118,1.583585,-0.472252,1.370770,1.891134,0.162964,0.392234
3,-1.630426,-1.472574,0.324651,0.399442,-0.859699,-1.695076,0.208421,0.090980,1.386668,-2.688693,-1.079306,-1.072986,1.556704,-1.054809
4,-1.362239,-0.042370,1.134816,0.258515,1.505896,0.262392,-1.732171,-0.000590,-0.188671,0.344332,2.027407,2.603663,0.162964,-2.019505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,-1.094051,1.519649,0.324651,0.399442,-0.859699,0.495424,0.208421,0.243596,0.402081,0.402659,-1.079306,-1.072986,1.556704,-1.054809
1104,0.783263,1.578967,-2.105844,-0.727974,-0.859699,-0.343491,0.208421,-0.366868,0.402081,0.402659,0.057496,0.038559,-0.568179,0.392234
1105,0.783263,-0.648724,0.324651,-0.727974,-1.253965,-0.669736,0.208421,-0.824717,-1.370175,-1.580472,-1.079306,-1.072986,-0.933750,-2.019505
1106,0.246888,-1.795523,0.324651,0.399442,-0.465433,-1.555257,0.208421,0.228335,0.402081,0.402659,-1.079306,-1.072986,-1.293610,-1.054809


In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_st, Y, test_size=1/5, random_state=42)

print(f'Train shape: {X_train.shape}, {y_train.shape}')
print(f'Test shape: {X_test.shape}, {y_test.shape}')

Train shape: (886, 14), (886,)
Test shape: (222, 14), (222,)


**Алгоритмы регрессии**

**Linear regression**

In [33]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)
  
print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squared Error:', mean_squared_error(y_test, pred))
print('R2 score:', r2_score(y_test, pred))

Mean Absolute Error: 265.5104597296495
Mean Squared Error: 144887.6645682123
R2 score: 0.7044148588882273


**KNN**

In [34]:
from sklearn.neighbors import KNeighborsRegressor 
     
scores = []
for i in range(1,25):
    score = {}
    knn = KNeighborsRegressor(i)
    knn.fit(X_train,y_train)
    answers = knn.predict(X_test)
    
    score['k'] = i
    score['MSE'] = mean_squared_error(y_test, answers)
    score['MAE'] = mean_absolute_error(y_test, answers)
    score['R2'] = r2_score(y_test, answers)
    
    scores.append(score)

scores = pd.DataFrame(scores)
scores

,k,MSE,MAE,R2
0,1,142000.322668,238.944324,0.710305
1,2,112610.291060,217.258581,0.770264
2,3,117967.649494,221.863393,0.759334
3,4,114883.877280,230.306036,0.765626
4,5,122186.262278,235.814568,0.750728
5,6,116592.522059,235.928949,0.762140
6,7,118625.867631,238.316403,0.757992
7,8,117099.977000,239.639949,0.761104
8,9,113353.466213,238.197482,0.768748
9,10,114667.875960,238.428351,0.766066


In [35]:
modelknn = KNeighborsRegressor(4)
modelknn.fit(X,Y)
prediction = modelknn.predict(X_test)
     

print(mean_squared_error(y_test,prediction))
print(mean_absolute_error(y_test,prediction))
print(r2_score(y_test, prediction))

984586.461273536
735.4847072072072
-1.0086535935242726


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


**TREE**

In [36]:
from sklearn.tree import DecisionTreeRegressor
     

scores = []
for i in range(1,40):
    score = {}
    clf = DecisionTreeRegressor(max_depth=i, random_state=42)
    clf.fit(X_train, y_train)
    answers = clf.predict(X_test)
    
    score['k'] = i
    score['MSE'] = mean_squared_error(y_test, answers)
    score['MAE'] = mean_absolute_error(y_test, answers)
    score['R2'] = r2_score(y_test, answers)
    
    scores.append(score)

scores = pd.DataFrame(scores)
scores

,k,MSE,MAE,R2
0,1,314465.344659,436.539547,0.358460
1,2,233980.980192,346.182080,0.522656
2,3,197896.859824,302.110749,0.596271
3,4,159622.873300,277.761368,0.674354
4,5,153488.967713,267.490955,0.686867
5,6,154809.552873,258.392052,0.684173
6,7,155208.005167,251.129887,0.683360
7,8,131817.338160,224.550104,0.731080
8,9,156740.415912,226.358683,0.680234
9,10,153592.545149,231.650873,0.686656


In [37]:
clf = DecisionTreeRegressor(random_state=42)
clf.fit(X_train, y_train)
y_pred_tree = clf.predict(X_test)

print(mean_squared_error(y_test,y_pred_tree))
print(mean_absolute_error(y_test,y_pred_tree))
print(r2_score(y_test, y_pred_tree))

163123.02337373624
239.82289039039034
0.6672129264682525


# **Работа с TEST**

In [38]:
test = pd.read_csv('/content/laptop-price-prediction-misis/Price_euros_test.csv')
test

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,86,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4GB,1TB HDD,AMD Radeon R5,Windows 10,2.1kg
1,1249,Razer,Blade Pro,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,1TB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95kg
2,122,Asus,VivoBook S15,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,256GB SSD,Nvidia GeForce 940MX,Windows 10,1.7kg
3,993,Asus,Rog GL753VE-DS74,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1050 Ti,Windows 10,2.99kg
4,739,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,12GB,1TB HDD,Intel HD Graphics 620,Windows 10,2.25kg
...,...,...,...,...,...,...,...,...,...,...,...,...
190,876,HP,Probook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,500GB HDD,Intel HD Graphics 520,Windows 10,2.07kg
191,282,Acer,Aspire A515-51G,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,1TB HDD,Nvidia GeForce MX130,Windows 10,2.2kg
192,528,HP,Spectre X360,Ultrabook,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.32kg
193,1204,Dell,Inspiron 5578,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,16GB,512GB SSD,Intel HD Graphics 620,Windows 10,2.09kg


In [39]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         195 non-null    int64  
 1   Company           195 non-null    object 
 2   Product           195 non-null    object 
 3   TypeName          195 non-null    object 
 4   Inches            195 non-null    float64
 5   ScreenResolution  193 non-null    object 
 6   Cpu               195 non-null    object 
 7   Ram               195 non-null    object 
 8   Memory            195 non-null    object 
 9   Gpu               195 non-null    object 
 10  OpSys             183 non-null    object 
 11  Weight            191 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 18.4+ KB


In [40]:
test.isna().sum()

laptop_ID            0
Company              0
Product              0
TypeName             0
Inches               0
ScreenResolution     2
Cpu                  0
Ram                  0
Memory               0
Gpu                  0
OpSys               12
Weight               4
dtype: int64

In [41]:
test= test.drop(['laptop_ID'], axis = 1)

In [42]:
test['Ram'] = test['Ram'].str.replace(r"[^\d\.]", "", regex=True)
test['Ram'] = pd.to_numeric(test['Ram'])

test['Weight'] = test['Weight'].str.replace(r"[^\d\.]", "", regex=True)
test['Weight'] = pd.to_numeric(test['Weight'])

test.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4,1TB HDD,AMD Radeon R5,Windows 10,2.10
1,Razer,Blade Pro,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,1TB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95
2,Asus,VivoBook S15,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8,256GB SSD,Nvidia GeForce 940MX,Windows 10,1.70
3,Asus,Rog GL753VE-DS74,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1050 Ti,Windows 10,2.99
4,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,12,1TB HDD,Intel HD Graphics 620,Windows 10,2.25


In [43]:
#заполним пропуски в ScreenResolution для дальнейшей работы
value111 = test['ScreenResolution'].mode()[0] #заполняем пропуски самыым часттым значением
test['ScreenResolution'] = test['ScreenResolution'].fillna(value111)

In [44]:
#разделяем Cpu на Cpu_Hz и Cpu_Model
test['Cpu_Hz'] = test['Cpu'].apply(lambda x: x.split()[-1])
test['Cpu_Hz'] = test['Cpu_Hz'].str.replace(r"[^\d\.]", "", regex=True)
test['Cpu_Hz'] = pd.to_numeric(test['Cpu_Hz'])

test['Cpu_Model'] = test['Cpu'].apply(lambda x: ' '.join(x.split()[:-1]))

#разделяем ScreenResolution на Resolution (который разделим на 2 числовых значения ширина(width) и высота(height)), 
#а модель Screen отбросим так как она есть далеко не везде, а пропуски нам не нужны.
test['ScreenResolution'] = test['ScreenResolution'].astype(str)
test['Resolution'] = test['ScreenResolution'].apply(lambda x: x.split()[-1])

test['Width'] = test['Resolution'].apply(lambda x: x.split('x')[0])
test['Width'] = pd.to_numeric(test['Width'])

test['Height'] = test['Resolution'].apply(lambda x: ' '.join(x.split('x')[1:]))
test['Height'] = pd.to_numeric(test['Height'])


#разделяем Memory на Memory_size и Memory_type
test['Memory_Size'] = test['Memory'].apply(lambda x: x.split()[0])
test['Memory_Size'] = test['Memory_Size'].str.replace(r"[^\d\.]", "", regex=True)
test['Memory_Size'] = pd.to_numeric(test['Cpu_Hz'])

test['Memory_type'] = test['Memory'].apply(lambda x: ' '.join(x.split()[1:]))

#удаляем колонки Memory, Cpu, ScreenResolution.
test = test.drop(['Memory'], axis = 1)
test = test.drop(['Cpu'], axis = 1)                   #удаляем столбцы которые мы уже разделили
test = test.drop(['ScreenResolution'], axis = 1)
test = test.drop(['Resolution'], axis = 1) 
test 

,Company,Product,TypeName,Inches,Ram,Gpu,OpSys,Weight,Cpu_Hz,Cpu_Model,Width,Height,Memory_Size,Memory_type
0,Acer,Aspire 3,Notebook,15.6,4,AMD Radeon R5,Windows 10,2.10,3.0,AMD A9-Series 9420,1366,768,3.0,HDD
1,Razer,Blade Pro,Gaming,14.0,16,Nvidia GeForce GTX 1060,Windows 10,1.95,2.8,Intel Core i7 7700HQ,1920,1080,2.8,SSD
2,Asus,VivoBook S15,Notebook,15.6,8,Nvidia GeForce 940MX,Windows 10,1.70,1.8,Intel Core i7 8550U,1920,1080,1.8,SSD
3,Asus,Rog GL753VE-DS74,Gaming,17.3,16,Nvidia GeForce GTX 1050 Ti,Windows 10,2.99,2.8,Intel Core i7 7700HQ,1920,1080,2.8,SSD + 1TB HDD
4,Dell,Inspiron 3567,Notebook,15.6,12,Intel HD Graphics 620,Windows 10,2.25,2.5,Intel Core i5 7200U,1366,768,2.5,HDD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,HP,Probook 450,Notebook,15.6,4,Intel HD Graphics 520,Windows 10,2.07,2.3,Intel Core i5 6200U,1920,1080,2.3,HDD
191,Acer,Aspire A515-51G,Notebook,15.6,8,Nvidia GeForce MX130,Windows 10,2.20,1.8,Intel Core i7 8550U,1920,1080,1.8,HDD
192,HP,Spectre X360,Ultrabook,13.3,8,Intel HD Graphics 620,Windows 10,1.32,2.7,Intel Core i7 7500U,1920,1080,2.7,SSD
193,Dell,Inspiron 5578,2 in 1 Convertible,15.6,16,Intel HD Graphics 620,Windows 10,2.09,2.7,Intel Core i7 7500U,1920,1080,2.7,SSD


In [45]:
value222 = test['OpSys'].mode()[0]
value333 = test['Weight'].mode()[0]

test['OpSys'] = test['OpSys'].fillna(value222)
test['Weight'] = test['Weight'].fillna(value333)

In [46]:
#test = pd.get_dummies(test, columns = ['Company','TypeName','OpSys','Memory_type']) 

test['Company'] = LabelEncoder().fit_transform(test['Company'])
test['TypeName'] = LabelEncoder().fit_transform(test['TypeName'])
test['OpSys'] = LabelEncoder().fit_transform(test['OpSys'])
test['Memory_type'] = LabelEncoder().fit_transform(test['Memory_type'])

test['Product'] = LabelEncoder().fit_transform(test['Product'])
test['Gpu'] = LabelEncoder().fit_transform(test['Gpu'])
test['Cpu_Model'] = LabelEncoder().fit_transform(test['Cpu_Model'])

test

,Company,Product,TypeName,Inches,Ram,Gpu,OpSys,Weight,Cpu_Hz,Cpu_Model,Width,Height,Memory_Size,Memory_type
0,0,11,3,15.6,4,4,4,2.10,3.0,5,1366,768,3.0,1
1,12,17,1,14.0,16,34,4,1.95,2.8,31,1920,1080,2.8,3
2,2,137,3,15.6,8,30,4,1.70,1.8,34,1920,1080,1.8,3
3,2,103,1,17.3,16,32,4,2.99,2.8,31,1920,1080,2.8,5
4,4,54,3,15.6,12,22,4,2.25,2.5,19,1366,768,2.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,5,94,3,15.6,4,19,4,2.07,2.3,17,1920,1080,2.3,1
191,0,13,3,15.6,8,45,4,2.20,1.8,34,1920,1080,1.8,1
192,5,106,4,13.3,8,22,4,1.32,2.7,29,1920,1080,2.7,3
193,4,59,0,15.6,16,22,4,2.09,2.7,29,1920,1080,2.7,3


In [47]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
test_std = scaler.fit_transform(test)
pd.DataFrame(test_std, columns = test.columns)

,Company,Product,TypeName,Inches,Ram,Gpu,OpSys,Weight,Cpu_Hz,Cpu_Model,Width,Height,Memory_Size,Memory_type
0,-1.772915,-1.441147,0.394976,0.461410,-0.876417,-1.929353,0.068966,0.077540,1.379441,-1.926299,-1.056886,-1.058035,1.379441,-1.126265
1,2.155704,-1.308171,-1.145431,-0.635009,1.459364,1.077231,0.068966,-0.142916,0.976338,1.017142,0.042074,0.022448,0.976338,0.299850
2,-1.118146,1.351360,0.394976,0.461410,-0.097823,0.676353,0.068966,-0.510342,-1.039180,1.356770,0.042074,0.022448,-1.039180,0.299850
3,-1.118146,0.597826,-1.145431,1.626355,1.459364,0.876792,0.068966,1.385577,0.976338,1.017142,0.042074,0.022448,0.976338,1.725964
4,-0.463376,-0.488149,0.394976,0.461410,0.680770,-0.125403,0.068966,0.297996,0.371682,-0.341369,-1.056886,-1.058035,0.371682,-1.126265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,-0.135991,0.398361,0.394976,0.461410,-0.876417,-0.426061,0.068966,0.033449,-0.031421,-0.567788,0.042074,0.022448,-0.031421,-1.126265
191,-1.772915,-1.396822,0.394976,0.461410,-0.097823,2.179645,0.068966,0.224510,-1.039180,1.356770,0.042074,0.022448,-1.039180,-1.126265
192,-0.135991,0.664314,1.165180,-1.114693,-0.097823,-0.125403,0.068966,-1.068829,0.774786,0.790723,0.042074,0.022448,0.774786,0.299850
193,-0.463376,-0.377335,-1.915635,0.461410,1.459364,-0.125403,0.068966,0.062843,0.774786,0.790723,0.042074,0.022448,0.774786,0.299850


In [48]:
answers = model.predict(test_std)
#answers = clf.predict(test)
#answers = modelknn.predict(test_std)


dfres = pd.DataFrame(answers, columns = ['Price_euros'])
dfres = dfres.reset_index()
dfres.to_csv("resultlaptop9.csv", index = False)
     

result = pd.read_csv('/content/resultlaptop9.csv')
result

,index,Price_euros
0,0,464.980586
1,1,1950.984851
2,2,964.917272
3,3,1761.553616
4,4,1088.069004
...,...,...
190,190,681.554883
191,191,949.870986
192,192,1413.807515
193,193,1601.516771


Итог: лучший MSE на кагле 66672.36921, достигнут с помощью линейной регрессии.
На 3.05.23 это 4 место из 16